In [3]:
import os
import shutil
import re
from fnmatch import fnmatch
from pathlib import Path
import csv
import pandas as pd
import numpy as np

In [4]:
def filter_useless_dir(dir_name: str):
    return True if re.match('[A-D].*', dir_name) and not re.match('A.*-A.*', dir_name) else False

def filter_useless_pic(pic_name: str):
    return True if re.match('[A-D].*[T|N].*', pic_name) else False

def normalize_pic_name(old_name):
    return old_name + ".dcm" if re.match('.*[T|N]$', old_name) else old_name




In [23]:

def walk_directory(datapath):
    """
        遍历文件夹下的文件，同时筛选出符合要求的文件夹
    """
    base_path = datapath
    if not os.path.exists(base_path):
        print("path {} not exist!".format(base_path))
        return -1
    dir_path_list, pic_path_list = [], []
    a, b  = 0, 0
    for root, dirs, files in os.walk(base_path, topdown=True):
        for name in files:
            a += 1
            if filter_useless_pic(name) is True:
                # normalization the picname
                old_name = name
                new_name = normalize_pic_name(name)
                old_tmp_path = os.path.join(root, old_name)
                new_tmp_path = os.path.join(root, new_name)
                os.rename(old_tmp_path, new_tmp_path)
                pic_path_list.append(new_tmp_path)
                print(old_name)
            else:
                # delete useless files
                os.remove(os.path.join(root, name))
                pass
                
        for name in dirs:
            b += 1
            if filter_useless_dir(name) is True:
                tmp_path = os.path.join(root, name)
                dir_path_list.append(tmp_path)
                print(tmp_path)
    print(a, b )
    return pic_path_list, dir_path_list
    

In [22]:
original_dataset_path = "../../multi-center"
dict = walk_directory(original_dataset_path)

dict

0 0


([], [])

In [16]:
pic_path_list, dir_path_list = dict[0], dict[1]

In [17]:
len(dir_path_list)
# len(pic_path_list)
# print(dir_path_list)
# print(pic_path_list)

112

In [18]:
def mv2traindir(srclist, dst):
    """
        移动前续遍历文件夹到正式的训练集/测试集文件夹下
    """
    a = 0 
    for src in srclist:
        src_name = os.path.split(src)[1]
        dst_path = os.path.join(dst, src_name)
        if os.path.exists(dst_path) is False:
            print(dst_path)
            shutil.move(src, dst_path)
            a += 1
    print(a)

def mv2testdir(srclist, dst):
    """
        移动前续遍历文件夹到正式的训练集/测试集文件夹下
    """
    a = 0 
    for src in srclist:
        src_name = os.path.split(src)[1]
        dst_path = os.path.join(dst, src_name)
        if os.path.exists(dst_path) is False:
            print(dst_path)
            shutil.move(src, dst_path)
            a += 1
    print(a)
    

In [19]:
# train_dataset = "../../bags_dataset/"
test_datatset = "../../bags_test/"

In [24]:
# mv2traindir(dir_path_list, train_dataset)
mv2testdir(dir_path_list, test_datatset)

../../bags_test/B00019-R3(3_11)R4(5_9)R6(8_8)


FileNotFoundError: [Errno 2] No such file or directory: '../../multi-center/B00001-B00036/B00019-R3(3_11)R4(5_9)R6(8_8)'

In [38]:
def walk_delete_emptydir(src):
    a = 0
    for root, dirs, names in os.walk(src):
        for name in dirs:
            # print(name)
            if len(os.listdir(os.path.join(root, name))) == 0:
                print(name)
                os.rmdir(os.path.join(root, name))
        for name in names: 
            a += 1          
    print(a)

def convertdcm2bmp(src):
    a = 0
    for root, dirs, names in os.walk(src):
        for name in names:
            # print(name)
            root_ = root
            root = root.replace("(", "").replace(")", "")
            os.rename(root_, root)
            # print(root)
            path = os.path.join(root, name)
            old_name = path
            new_name = old_name[:-4] + ".bmp"
            if re.match(".*.[dcm]$", name):
                # print(old_name, " => ", new_name)
                # os.system(f"dcmj2pnm --write-bmp {old_name} {new_name}")
                os.remove(old_name)
            if re.match(".*.[tiff]$", name):
                old_name = path
                new_name = old_name[:-5] + ".jpg"
                os.system(f"convert {old_name} {new_name}")
                os.remove(old_name)
                
            if re.match(".*-0_.*", name):
                # print(new_name) 
                nn_name = new_name.replace("-0_", "-P0_")
                print(nn_name)
                os.rename(new_name, nn_name)


In [40]:
walk_delete_emptydir(test_datatset)

440


In [41]:
convertdcm2bmp(test_datatset)

In [42]:
walk_delete_emptydir(test_datatset)


369


In [43]:
# annotation_file_path = "../mil_annontations/data.csv"
# instance_label = "../mil_annotations/label.csv"
# idx_label = "../mil_annotations/idx.csv"
# train_dataset = "../../bags_dataset/"

# os.mkdir("../mil_annontations")

# def generate_file_path(src, dst):
#     csv_dict = {}


def generate_instance_label(src, dst):
    """
    instance level labels
    """
    label_dict = {}
    n = ['img_path', 'label']
    for root, dirs, names in os.walk(src):
        for name in names:
            path = os.path.join(root, name)
            if re.match(".*-P0_.*", name):
                label_dict[path] = 0
            # elif re.match(".*-P[0-9]_.*", name):
            #     label_dict[path] = 1
            else:
                label_dict[path] = 1
    try: 
        with open(dst, 'w') as f:
            w = csv.writer(f)
            for k,v in label_dict.items():
                w.writerow([k, v])
    except OSError as e:
        print(e.strerror)
    #add_header
    df = pd.read_csv(dst, names=n)
    df.to_csv(dst, index=False)
    print("instance level csv generation complete!")
    # print(label_dict)
                
def generate_idx(src, dst):
    label_dict = {}
    idx = 0 # iteratively increase to distinguish different bags
    n = ['img_path', 'idx']
    pre = tmp = 0
    for root, dirs, names in os.walk(src):
        for name in names:
            in_directory = os.path.split(root)[1]
            # print(in_directory, "->", name)
            pre = tmp
            tmp = in_directory
            if pre == 0:
                label_dict[os.path.join(root, name)] = idx
            elif tmp == pre:
                label_dict[os.path.join(root, name)] = idx
            else:
                idx += 1 
                label_dict[os.path.join(root, name)] = idx               
    try: 
        with open(dst, 'w') as f:
            w = csv.writer(f)
            for k,v in label_dict.items():
                w.writerow([k,v])
    except OSError as e:
        print(e.strerror)
    #add_header
    df = pd.read_csv(dst, names=n)
    df.to_csv(dst, index=False)
    print("idx csv generation complete!")
            

def generate_bags_annotations(src, dst):
    pass

    


In [44]:
instance_label = "../mil_annotations/label_test.csv"
idx_label = "../mil_annotations/idx_test.csv"
generate_instance_label(test_datatset, instance_label)
generate_idx(test_datatset, idx_label)

instance level csv generation complete!
idx csv generation complete!


SyntaxError: invalid syntax (170990008.py, line 1)